## LSSTCam Astrometry learning

Craig Lage - 05-May-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler
from lsst.summit.utils.plotting import plot
import lsst.afw.image as afwImage
from lsst.geom import SpherePoint
from lsst.geom import Angle as afwAngle
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.afw import cameraGeom
import lsst.geom
import lsst.geom as geom
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I

from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
from lsst.obs.lsst import LsstComCam
from os import listdir
import healpy as hp
from astropy.table import Table, vstack
from astropy.coordinates import angular_separation


In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])

In [ ]:
expId = 2025042500591
calexp = butler.get('preliminary_visit_image', detector=94, visit=expId)

In [ ]:
x = plot(calexp, stretch='ccs')

In [ ]:
def get_stars(detector,ra,dec,cam_wcs,npixedge=0):
    # some constants and useful objects
    ccd_diag = 0.15852  #Guider CCD diagonal radius in Degrees
    path = '/home/s/shuang92/rubin-user/Monster_guide'  ## RSP path
    res = 5
    nside = 2**res
    npix = 12 * nside**2
    bad_guideramps = {193: 'C1', 198: 'C1', 201: 'C0'}
    filters = ['u','g','r','i','z','y']

    # get BBox for this detecctor, removing some number of edge pixels
    ccd_bbox = detector.getBBox()
    ccd_bbox.grow(-Extent2I(npixedge, npixedge))
    
    # query the Monster
    hp_ind = hp.ang2pix(nside, ra, dec, lonlat=True)

    # should only need at most 4 tiles, but for simplicity using 9 Tables
    SW, W, NW, N, NE, E, SE, S = hp.get_all_neighbours(nside, hp_ind)
    
    this_table = Table.read(f'{path}/{hp_ind}.csv')

    E_table = Table.read(f'{path}/{E}.csv')
    W_table = Table.read(f'{path}/{W}.csv')
    S_table = Table.read(f'{path}/{S}.csv')
    N_table = Table.read(f'{path}/{N}.csv')

    SW_table = Table.read(f'{path}/{SW}.csv')
    SE_table = Table.read(f'{path}/{SE}.csv')
    NW_table = Table.read(f'{path}/{NW}.csv')
    NE_table = Table.read(f'{path}/{NE}.csv')

    star_cat = vstack([this_table, E_table, W_table, S_table, N_table, SW_table, SE_table, NW_table, NE_table])

    # find the ra,dec of the CCD center
    x0,y0 = detector.getBBox().getCenterX(),detector.getBBox().getCenterY()  
    ra_ccd,dec_ccd = cam_wcs.pixelToSky(x0,y0)

    star_cat['dangle'] = np.degrees(angular_separation(ra_ccd.asRadians(),dec_ccd.asRadians(),
                            star_cat['coord_ra'],star_cat['coord_dec']))
    insideCCDradius = (star_cat['dangle']<ccd_diag) # inside the Guider CCD radius

    # Selection 1: the Star is isolated and inside the CCD radius
    cat_select1 = star_cat[(insideCCDradius)]

    # using the wcs to locate the stars inside the CCD minus npixedge
    ccdx,ccdy = cam_wcs.skyToPixelArray(cat_select1['coord_ra'],cat_select1['coord_dec'],degrees=False)
    inCCD = ccd_bbox.contains(ccdx,ccdy)

    # fill with CCD pixel x,y
    cat_select1['ccdx'] = ccdx
    cat_select1['ccdy'] = ccdy
    cat_select1['inCCD'] = inCCD
    

    return cat_select1

In [ ]:
camera = LsstCam.getCamera()
wcs = calexp.getWcs()
center = wcs.pixelToSky(2048, 2000)
ra_center = center.getRa().asDegrees()
dec_center = center.getDec().asDegrees()

raft = 'R22'
ccd = 'S11'
detName = f"{raft}_{ccd}"
for detector in camera:
    if detector.getName()== detName:
        break

cat_select1 = get_stars(detector,ra_center,dec_center,wcs,npixedge=0)
xs = []
ys = []
mags = []
for i in range(len(cat_select1)):
    x = cat_select1['ccdx'][i]
    y = cat_select1['ccdy'][i]
    if cat_select1['inCCD'][i]:
        xs.append(x)
        ys.append(y)
        mags.append(f"{cat_select1['mag_r'][i]:.1f}")

In [ ]:
# Now plot the data with matplotlib
%matplotlib inline
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(16,16))
plt.title(f"{expId} {raft}_{ccd}")
img = plt.imshow(calexp.image.array,  interpolation='Nearest', cmap='gray', vmin=0, vmax=1000, origin='lower')
plt.scatter(xs, ys\
            ,facecolors='none', edgecolors='g', s=200, lw=2)
for x, y, mag in zip(xs, ys, mags):
    plt.text(x+50,y, mag, color='g')
colorbar(img)


In [ ]:
def get_stars_focal_plane(detector,ra,dec,cam_wcs,npixedge=0):
    # some constants and useful objects
    ccd_diag = 0.15852  #Guider CCD diagonal radius in Degrees
    path = '/home/s/shuang92/rubin-user/Monster_guide'  ## RSP path
    res = 5
    nside = 2**res
    npix = 12 * nside**2
    bad_guideramps = {193: 'C1', 198: 'C1', 201: 'C0'}
    filters = ['u','g','r','i','z','y']
    
    # query the Monster
    hp_ind = hp.ang2pix(nside, ra, dec, lonlat=True)

    # should only need at most 4 tiles, but for simplicity using 9 Tables
    SW, W, NW, N, NE, E, SE, S = hp.get_all_neighbours(nside, hp_ind)
    
    this_table = Table.read(f'{path}/{hp_ind}.csv')

    E_table = Table.read(f'{path}/{E}.csv')
    W_table = Table.read(f'{path}/{W}.csv')
    S_table = Table.read(f'{path}/{S}.csv')
    N_table = Table.read(f'{path}/{N}.csv')

    SW_table = Table.read(f'{path}/{SW}.csv')
    SE_table = Table.read(f'{path}/{SE}.csv')
    NW_table = Table.read(f'{path}/{NW}.csv')
    NE_table = Table.read(f'{path}/{NE}.csv')

    star_cat = vstack([this_table, E_table, W_table, S_table, N_table, SW_table, SE_table, NW_table, NE_table])

    # find the ra,dec of the CCD center
    bbox = detector.getBBox()
    nx,ny = bbox.getDimensions()
    detName = detector.getName()
    lct = LsstCameraTransforms(camera,detName)
    llfpX, llfpY = lct.ccdPixelToFocalMm(0, 0, detName)
    llfpX = llfpX * 100 + int(nx/2) # Convert to pixels and recenter
    llfpY = llfpY * 100 + int(ny/2) # Convert to pixels and recenter
    urfpX, urfpY = lct.ccdPixelToFocalMm(nx, ny, detName)
    urfpX = urfpX * 100 + int(nx/2) # Convert to pixels and recenter
    urfpY = urfpY * 100 + int(ny/2) # Convert to pixels and recenter
    x0 = int((urfpX + llfpX) / 2) # In pixels
    y0 = int((urfpY + llfpY) / 2) # In pixels
    ra_ccd,dec_ccd = cam_wcs.pixelToSky(x0,y0)

    ccd_bbox = Box2I(Point2I(llfpX, llfpY), 
                     Point2I(urfpX, urfpY))
    star_cat['dangle'] = np.degrees(angular_separation(ra_ccd.asRadians(),dec_ccd.asRadians(),
                            star_cat['coord_ra'],star_cat['coord_dec']))
    insideCCDradius = (star_cat['dangle']<ccd_diag) # inside the Guider CCD radius

    # Selection 1: the Star is isolated and inside the CCD radius
    cat_select1 = star_cat[(insideCCDradius)]

    # using the wcs to locate the stars inside the CCD minus npixedge
    ccdx,ccdy = cam_wcs.skyToPixelArray(cat_select1['coord_ra'],cat_select1['coord_dec'],degrees=False)
    inCCD = ccd_bbox.contains(ccdx,ccdy)

    # fill with CCD pixel x,y
    cat_select1['ccdx'] = ccdx - llfpX
    cat_select1['ccdy'] = ccdy - llfpY
    cat_select1['inCCD'] = inCCD
    

    return cat_select1

In [ ]:
expId = 2025042500591
calexp2 = butler.get('preliminary_visit_image', detector=36, visit=expId)

In [ ]:
x = plot(calexp2, stretch='ccs')

In [ ]:
raft2 = 'R11'
ccd2 = 'S00'
detName = f"{raft2}_{ccd2}"
for detector2 in camera:
    if detector2.getName()== detName:
        break
print(detector2.getName())

wcs2 = calexp2.getWcs()
center2 = wcs2.pixelToSky(2048, 2000)
ra_center2 = center2.getRa().asDegrees()
dec_center2 = center2.getDec().asDegrees()


cat_select2 = get_stars_focal_plane(detector2,ra_center2,dec_center2,wcs,npixedge=0)
xs = []
ys = []
mags = []
for i in range(len(cat_select2)):
    x = cat_select2['ccdx'][i]
    y = cat_select2['ccdy'][i]
    if cat_select2['inCCD'][i]:
        xs.append(x)
        ys.append(y)
        mags.append(f"{cat_select2['mag_r'][i]:.1f}")


In [ ]:
# Now plot the data with matplotlib
%matplotlib inline
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(16,16))
plt.title(f"{expId} {raft2}_{ccd2}")
img = plt.imshow(calexp2.image.array,  interpolation='Nearest', cmap='gray', vmin=0, vmax=1000, origin='lower')
plt.scatter(xs, ys\
            ,facecolors='none', edgecolors='g', s=200, lw=2)
for x, y, mag in zip(xs, ys, mags):
    plt.text(x+50,y, mag, color='g')
colorbar(img)


In [ ]:
raft2 = 'R40'
ccd2 = 'SG0'
detName = f"{raft2}_{ccd2}"
for detector2 in camera:
    if detector2.getName()== detName:
        break
print(detector2.getName())

wcs2 = calexp2.getWcs()
center2 = wcs2.pixelToSky(2048, 2000)
ra_center2 = center2.getRa().asDegrees()
dec_center2 = center2.getDec().asDegrees()


cat_select2 = get_stars_focal_plane(detector2,ra_center2,dec_center2,wcs,npixedge=0)
xs = []
ys = []
mags = []
for i in range(len(cat_select2)):
    x = cat_select2['ccdx'][i]
    y = cat_select2['ccdy'][i]
    if cat_select2['inCCD'][i]:
        xs.append(x)
        ys.append(y)
        mags.append(f"{cat_select2['mag_r'][i]:.1f}")

In [ ]:
# Now plot the data with matplotlib
%matplotlib inline
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(16,16))
plt.title(f"{expId} {raft2}_{ccd2}")
dummy = np.zeros([4096, 4000])
img = plt.imshow(dummy,  interpolation='Nearest', cmap='gray', vmin=0, vmax=1000, origin='lower')
plt.scatter(xs, ys\
            ,facecolors='none', edgecolors='g', s=200, lw=2)
for x, y, mag in zip(xs, ys, mags):
    plt.text(x+50,y, mag, color='g')
colorbar(img)
